## Theory
This notebook tests some of the basic concepts of causal relationships and statistical modelling, as laid out in Pearl & MacKenzie (2018). For example, controlling for a "collider" will introduce bias where there was none. Subsequent notebooks will test out how well different meta-learners work, and what are the practical implications of accounting for/ ignoring causal relationships in predictive models.

### Approach
I will be using simulated datasets to lay out "true" causal relationships. For simplicity, I will be using linear regressions. To aid understanding, we will be trying to discern the effect of number of years of education on wages, with various confounding causal relationships

### Contents
1) Basic confounding - treatment and effect share a common cause
2) Collider
3) Mediation
4) Back-door path with collider

In [126]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

In [127]:
# global parameters

education_mean = 12
education_std = 3

wage_mean = 36000
wage_std = 3000

education_effect_wage = 1000

## 1. Basic confounding

education + family wealth ----> wage

family_wealth ----> education

Notes: 
- This is a standard omitted variable bias problem.
- Intuition: suppose that family_wealth contributes positively to an individual's level of education, and positively to wages. If we estimate the effect of education on wages and ignore family_wealth, then we will attribute some of the contribution stemming from family_wealth to education and over-estimate the effect of education.

In [128]:
def basic_confounding_dataset(
        sample_size: int = 1000
        , sample_education_mean:int = education_mean 
        , sample_wage_mean:int = wage_mean 
        , sample_education_effect_wage:int = education_effect_wage
        , sample_family_wealth_effect_wage:int = 5000
        , sample_family_wealth_effect_education:int = 1
        , **kwargs
):
    
    # create randomised data    
    _family_wealth = np.random.standard_normal(sample_size)
    _education = sample_education_mean + (sample_family_wealth_effect_education * _family_wealth) + (np.sqrt(3) * np.random.standard_normal(sample_size))
    _wage = (
        sample_wage_mean + (sample_education_effect_wage * (_education - sample_education_mean)) + (sample_family_wealth_effect_wage * _family_wealth) + (3000 * np.random.standard_normal(sample_size))
    )

    _df = pd.DataFrame({'family_wealth': _family_wealth, 'education': _education, 'wage': _wage})
    
    return _df



In [129]:
# create data
df1 = basic_confounding_dataset()
df1.describe()

,family_wealth,education,wage
count,1000.000000,1000.000000,1000.000000
mean,0.020233,12.001240,36157.015713
std,1.001601,1.921364,6892.623816
min,-3.718065,5.209940,11738.241724
25%,-0.716165,10.652876,31311.234502
50%,0.014908,12.023944,36382.309900
75%,0.710828,13.308900,40733.075904
max,2.933665,17.756259,59845.514690


In [130]:
# omitted variable bias
X = sm.add_constant(df1['education'])
y = df1['wage']

mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.399
Model:                            OLS   Adj. R-squared:                  0.398
Method:                 Least Squares   F-statistic:                     661.7
Date:                Sat, 13 Jul 2024   Prob (F-statistic):          2.38e-112
Time:                        21:33:22   Log-Likelihood:                -10002.
No. Observations:                1000   AIC:                         2.001e+04
Df Residuals:                     998   BIC:                         2.002e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       8973.1763   1070.238      8.384      0.0

In [131]:
# fully specified

X = sm.add_constant(df1[['education', 'family_wealth']])
y = df1['wage']

mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.800
Model:                            OLS   Adj. R-squared:                  0.800
Method:                 Least Squares   F-statistic:                     1997.
Date:                Sat, 13 Jul 2024   Prob (F-statistic):               0.00
Time:                        21:33:22   Log-Likelihood:                -9451.2
No. Observations:                1000   AIC:                         1.891e+04
Df Residuals:                     997   BIC:                         1.892e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          2.326e+04    694.771     33.486

## 2. Collider

education -----> liberal_support <----- wage

Notes:
- In this example suppose that education does not affect wage.
- Support for liberal politics is not on the causal path from education to wage; instead it is affected by both. Controlling for liberal_support will spuriously introduce a relationship between education and wage.
- Intuition: 

In [153]:
def collider_dataset(
        sample_size: int = 1000
        , sample_education_mean:int = education_mean 
        , sample_education_std:int = education_std
        , sample_wage_mean:int = wage_mean 
        , sample_education_effect_wage:int = 0
        , sample_education_effect_liberal_support:int = 1
        , sample_wage_effect_liberal_support:int = -1
        , **kwargs
):
    
    # create randomised data    
    _education = sample_education_mean + (sample_education_std * np.random.standard_normal(sample_size))
    _wage = (
        sample_wage_mean + (sample_education_effect_wage * (_education - sample_education_mean)) + (3000 * np.random.standard_normal(sample_size))
    )

    _education_normalised = (_education - np.mean(_education))/ np.std(_education)
    _wage_normalised = (_wage - np.mean(_wage))/ np.std(_wage)

    _liberal_support = (
        (sample_education_effect_liberal_support * _education_normalised) + (sample_wage_effect_liberal_support * _wage_normalised) + np.random.standard_normal(sample_size)
    )

    _df = pd.DataFrame({'liberal_support': _liberal_support, 'education': _education, 'wage': _wage})
    
    return _df

In [154]:
df2 = collider_dataset()
df2.describe()

,liberal_support,education,wage
count,1000.000000,1000.000000,1000.000000
mean,-0.009468,12.000307,35952.113290
std,1.689139,3.031342,2984.841577
min,-4.456029,3.159568,25277.542392
25%,-1.139316,9.853634,34040.616161
50%,0.003688,12.007941,35875.511844
75%,1.075675,13.996619,37841.992258
max,5.516947,21.079281,46674.655621


In [155]:
# "correct" regression - ignore collider

X = sm.add_constant(df2['education'])
y = df2['wage']

mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.2404
Date:                Sat, 13 Jul 2024   Prob (F-statistic):              0.624
Time:                        21:37:23   Log-Likelihood:                -9419.6
No. Observations:                1000   AIC:                         1.884e+04
Df Residuals:                     998   BIC:                         1.885e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3.577e+04    385.727     92.731      0.0

In [156]:
# incorrect - control for collider

X = sm.add_constant(df2[['education', 'liberal_support']])
y = df2['wage']

mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.505
Model:                            OLS   Adj. R-squared:                  0.504
Method:                 Least Squares   F-statistic:                     507.9
Date:                Sat, 13 Jul 2024   Prob (F-statistic):          7.87e-153
Time:                        21:37:26   Log-Likelihood:                -9068.5
No. Observations:                1000   AIC:                         1.814e+04
Df Residuals:                     997   BIC:                         1.816e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            2.987e+04    328.753     

## 3. Mediator

education ---> skills ---> wage

In [136]:
def mediator_dataset(
        sample_size: int = 1000
        , sample_education_mean:int = education_mean 
        , sample_education_std:int = education_std
        , sample_wage_mean:int = wage_mean 
        , sample_education_effect_wage: int = education_effect_wage
        , sample_skill_mean:int = 5
        , sample_education_effect_skill:int = 0.2
        , **kwargs
):
    
    # create randomised data    
    _education = sample_education_mean + (sample_education_std * np.random.standard_normal(sample_size))

    _skill = (
        sample_skill_mean + (sample_education_effect_skill * (_education - sample_education_mean)) + np.random.standard_normal(sample_size)
    )

    _implied_skill_effect_on_wage = sample_education_effect_wage / sample_education_effect_skill

    _wage = (
        sample_wage_mean + (_implied_skill_effect_on_wage * _skill) + (3000 * np.random.standard_normal(sample_size))
    )


    _df = pd.DataFrame({'skill': _skill, 'education': _education, 'wage': _wage})
    
    return _df

In [137]:
df3 = mediator_dataset()
df3.describe()

,skill,education,wage
count,1000.000000,1000.000000,1000.000000
mean,5.023052,12.020312,61177.809910
std,1.169367,3.055703,6547.784112
min,1.433315,2.876282,39736.054804
25%,4.190613,9.920015,56735.222555
50%,5.005359,12.227941,61261.577290
75%,5.811079,13.986940,65666.163400
max,8.801578,21.204344,85488.528289


In [138]:
# correct - do not control for mediator collider

X = sm.add_constant(df3['education'])
y = df3['wage']

mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.267
Model:                            OLS   Adj. R-squared:                  0.266
Method:                 Least Squares   F-statistic:                     362.7
Date:                Sat, 13 Jul 2024   Prob (F-statistic):           3.21e-69
Time:                        21:33:22   Log-Likelihood:                -10050.
No. Observations:                1000   AIC:                         2.010e+04
Df Residuals:                     998   BIC:                         2.011e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.788e+04    720.447     66.458      0.0

In [139]:
# incorrect - control for mediator

X = sm.add_constant(df3[['education', 'skill']])
y = df3['wage']

mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.801
Model:                            OLS   Adj. R-squared:                  0.800
Method:                 Least Squares   F-statistic:                     2002.
Date:                Sat, 13 Jul 2024   Prob (F-statistic):               0.00
Time:                        21:33:22   Log-Likelihood:                -9399.0
No. Observations:                1000   AIC:                         1.880e+04
Df Residuals:                     997   BIC:                         1.882e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3.586e+04    441.928     81.148      0.0

## 4. Backdoor/ "M bias"

education + ambition ----> wage

parents_education ----> education

parents_education ----> books_read_per_year <----- ambition


In [140]:
def m_bias_dataset(
        sample_size: int = 1000
        , sample_education_mean:int = education_mean 
        , sample_education_std:int = education_std
        , sample_wage_mean:int = wage_mean 
        , sample_education_effect_wage: int = education_effect_wage
        , sample_parents_effect_education: int = 0.5
        , sample_ambition_effect_wage: int = 5000
        , sample_books_read_mean: int = 5
        , parents_education_effect_books_read: int = 1
        , ambition_effect_books_read: int = 1
        , **kwargs
):
    
    # create randomised data    
    _parents_education_mean = sample_education_mean - 3
    _parents_education = _parents_education_mean + ((sample_education_std - 1) * np.random.standard_normal(sample_size))
    _ambition = np.random.standard_normal(sample_size)

    _education = (
        sample_education_mean 
        + (sample_parents_effect_education * (_parents_education - _parents_education_mean)) 
        + (sample_education_std * np.random.standard_normal(sample_size))
    )
    
    _wage = (
        sample_wage_mean 
        + (sample_education_effect_wage * (_education - sample_education_mean)) 
        + (sample_ambition_effect_wage * _ambition) + (3000 * np.random.standard_normal(sample_size))
    )

    _books_read = (
        sample_books_read_mean 
        + (parents_education_effect_books_read * (_parents_education - _parents_education_mean)) 
        + (ambition_effect_books_read * _ambition)
        + np.random.standard_normal(sample_size)
    )
    

    _df = pd.DataFrame({
        'education': _education, 'wage': _wage
        , 'parents_education': _parents_education
        , 'ambition': _ambition
        , 'books_read': _books_read
        })
    
    return _df

In [141]:
df4 = m_bias_dataset(**{'sample_education_effect_wage':1000})
df4.describe()

,education,wage,parents_education,ambition,books_read
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,11.884877,35949.315252,8.945845,0.028851,5.011202
std,3.255786,6864.394296,2.108077,1.021251,2.571658
min,1.042774,13793.740300,2.091096,-3.180104,-3.864299
25%,9.529323,31494.278946,7.539727,-0.683049,3.405838
50%,12.003117,36103.069263,8.938015,0.037976,5.058400
75%,14.153542,40395.553562,10.341510,0.710042,6.664085
max,22.333853,60110.622734,16.613146,3.076785,12.907795


In [142]:
# correct - ignore backdoor as already closed

X = sm.add_constant(df4['education'])
y = df4['wage']

mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.209
Model:                            OLS   Adj. R-squared:                  0.208
Method:                 Least Squares   F-statistic:                     263.8
Date:                Sat, 13 Jul 2024   Prob (F-statistic):           8.05e-53
Time:                        21:33:22   Log-Likelihood:                -10135.
No. Observations:                1000   AIC:                         2.027e+04
Df Residuals:                     998   BIC:                         2.028e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.449e+04    731.370     33.487      0.0

In [143]:
# correct - control for parents' education and ambition, but do not control for collider (books read)

X = sm.add_constant(df4[['education', 'parents_education', 'ambition']])
y = df4['wage']

mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.795
Model:                            OLS   Adj. R-squared:                  0.795
Method:                 Least Squares   F-statistic:                     1289.
Date:                Sat, 13 Jul 2024   Prob (F-statistic):               0.00
Time:                        21:33:22   Log-Likelihood:                -9459.8
No. Observations:                1000   AIC:                         1.893e+04
Df Residuals:                     996   BIC:                         1.895e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              2.347e+04    485.86

In [144]:
# incorrect - control for collider (books read), thereby opening the backdoor

X = sm.add_constant(df4[['education','books_read']])
y = df4['wage']

mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.302
Model:                            OLS   Adj. R-squared:                  0.301
Method:                 Least Squares   F-statistic:                     215.9
Date:                Sat, 13 Jul 2024   Prob (F-statistic):           1.25e-78
Time:                        21:33:22   Log-Likelihood:                -10073.
No. Observations:                1000   AIC:                         2.015e+04
Df Residuals:                     997   BIC:                         2.017e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.254e+04    707.785     31.848      0.0

In [145]:
# control - control for collider (books read) but also close the backdoor path by controlling for the other variables

X = sm.add_constant(df4[['education','books_read', 'ambition']])
y = df4['wage']

mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.795
Model:                            OLS   Adj. R-squared:                  0.794
Method:                 Least Squares   F-statistic:                     1288.
Date:                Sat, 13 Jul 2024   Prob (F-statistic):               0.00
Time:                        21:33:22   Log-Likelihood:                -9459.9
No. Observations:                1000   AIC:                         1.893e+04
Df Residuals:                     996   BIC:                         1.895e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.358e+04    384.306     61.363      0.0